In [1]:
:load src/stats.hs

In [2]:
import Text.CSV
import Stats
import System.Random
import Data.Maybe
import Data.List
import Data.Ord
import Graphics.EasyPlot

In [3]:
monet <- parseCSVFromFile "./data/monet.csv"

In [4]:
g <- newStdGen

In [5]:
values = take 100000 (randomRs (1,100) g) :: [Double]

In [6]:
take 10 values

[51.17858236198211,19.312874912357394,42.42218425987883,64.41077879589028,31.365459975711204,43.74471880353704,86.0879931932969,12.927120490316357,99.60039558399471,41.93950562538258]

In [7]:
chunk [] = []
chunk list = take 10 list : chunk (drop 10 list)

In [8]:
length $ chunk values

10000

In [9]:
avgs = map (\xs -> sum xs / genericLength xs) $ chunk values

In [10]:
pairs = runLengthEncoding $ sort $ map round avgs

In [11]:
plot X11 $ Data2D [Style Lines] [] (map (\(x,y) -> (fromIntegral x, y)) pairs)

True

In [12]:
normal mu sd x = exp (-((x-mu)^2 / (2*sd*sd))) / sqrt (2*sd*sd*pi)

In [13]:
domain = [-5,-4.9..5]

In [14]:
plot X11 $ Data2D [Style Lines] [] $ zip domain (map (normal 0 0.1) domain)

True

In [15]:
arr = [1,1,5]
range arr

Just (1,5)

In [16]:
domain = [-4,-3.9..10]

In [17]:
c1 = map (normal 1 1) domain
c2 = map (normal 5 1) domain

In [18]:
curves = [c1, c1, c2]

In [19]:
kde_ = foldl1 (zipWith (+)) curves
kdeAdj_ = map (/ (sum kde_)) kde_

In [20]:
plot X11 [Data2D [Style Lines] [] (zip domain kdeAdj_)]

True

In [21]:
kde :: [Double] -> Maybe ([Double], [Double])
kde [] = Nothing
kde xs = Just (domain, mykde)
    where
        mykde = map (/ (sum shape)) shape
        shape = foldl1 (zipWith (+)) $ map (\x -> (map (normal x 1) domain)) xs
        low = -5 + fst highLow
        high = 5 + snd highLow
        highLow = fromJust $ range xs
        domain = [low, (low+0.1)..high]

In [24]:
monetPrices :: [Double]
monetPrices = readIndex monet 0

In [25]:
plot X11 $ zip [1..] $ sort monetPrices

True

In [26]:
Just (domain, kdeAdj) = kde monetPrices

In [27]:
plot X11 $ zip domain kdeAdj

True

In [32]:
indices = findIndices (>= 5) domain

In [34]:
sum $ map (kdeAdj !!) indices

0.19975630172705675